In [12]:
# !pip install torch torchvision
# !pip install opencv-python
# !pip install --upgrade ultralytics


In [3]:
from ultralytics import YOLO

# Carregar o modelo YOLOv8 treinado
model = YOLO('runs/detect/train/weights/best.pt')

# Função para realizar a predição em uma imagem
def predict_image(image_path):
    results = model.predict(image_path)  # Faz a predição na imagem
    return results[0].boxes.xyxy, results[0].boxes.conf, results[0].boxes.cls

# Caminhos das imagens para análise
image_paths = ['img1.jpeg', 'img2.jpeg']

# Analisar cada imagem
for image_path in image_paths:
    boxes, scores, labels = predict_image(image_path)
    print(f"Resultados para {image_path}:")
    print("Boxes:", boxes)
    print("Scores:", scores)
    print("Labels:", labels)



image 1/1 /home/gamorab/Desktop/epib-box/model/v1/img1.jpeg: 608x800 1 bola, 1 bomba, 1 bota1, 1 bota2, 1 cx-bike, 1 cx-ferro, 1 cx-magali, 1 cx-minions, 1 cx-xuxa, 1 luz, 1 pote, 44.1ms
Speed: 2.2ms preprocess, 44.1ms inference, 107.0ms postprocess per image at shape (1, 3, 608, 800)
Resultados para img1.jpeg:
Boxes: tensor([[ 380.3805,  706.1035,  486.2802,  837.9971],
        [ 684.5367,  543.6744, 1082.8741,  829.1696],
        [ 752.3357,  340.6420,  888.7118,  474.2230],
        [ 147.8054,   50.4468,  599.5676,  467.7499],
        [1063.2378,   45.3196, 1138.1826,  490.6250],
        [1345.4351,  274.8486, 1537.2952,  486.4080],
        [ 539.4995,  984.9575,  805.6143, 1191.0684],
        [1243.0078,  986.4401, 1410.2544, 1194.8385],
        [1246.7371,  735.4508, 1384.2468,  848.5428],
        [ 309.2712, 1098.4216,  386.1266, 1187.9363],
        [ 979.2996,  985.6356, 1082.2258, 1190.3544]], device='cuda:0')
Scores: tensor([0.9864, 0.9744, 0.9726, 0.9655, 0.9603, 0.8926, 0.8

In [7]:
import json
from collections import Counter

# Mapeamento das classes (extraído do arquivo data.yaml)
class_names = ['bola', 'bomba', 'bota1', 'bota2', 'capacete', 'cx-bike', 'cx-ferro', 'cx-magali', 'cx-minions', 'cx-xuxa', 'isqueiro', 'luz', 'pote']

# Função para realizar a predição e contar as classes
def predict_and_count(image_path):
    # Realizar a predição
    results = model.predict(image_path)
    
    # Obter os labels (classes detectadas) da primeira imagem
    labels = results[0].boxes.cls.cpu().numpy().astype(int)  # Converter para inteiros
    
    # Contar quantas vezes cada label foi detectado
    label_counts = Counter(labels)
    
    # Mapear os índices para os nomes das classes
    counts_with_names = {class_names[label]: count for label, count in label_counts.items()}
    
    return counts_with_names

# Função para calcular a diferença entre as imagens
def calculate_difference(counts_img1, counts_img2):
    diff_img1 = {label: count for label, count in counts_img1.items() if label not in counts_img2}
    diff_img2 = {label: count for label, count in counts_img2.items() if label not in counts_img1}
    return diff_img1, diff_img2

# Caminhos das imagens para análise
image_paths = ['img1.jpeg', 'img2.jpeg']

# Obter as contagens de objetos para cada imagem
counts_img1 = predict_and_count(image_paths[0])
counts_img2 = predict_and_count(image_paths[1])

# Calcular a diferença entre as imagens
diff_img1, diff_img2 = calculate_difference(counts_img1, counts_img2)

# Gerar o JSON com as informações
output = {
    "image1": {
        "path": image_paths[0],
        "objects": counts_img1
    },
    "image2": {
        "path": image_paths[1],
        "objects": counts_img2
    },
    "difference": {
        "in_image1_not_in_image2": diff_img1,
        "in_image2_not_in_image1": diff_img2
    }
}

# Salvar o JSON em um arquivo
with open('output.json', 'w') as json_file:
    json.dump(output, json_file, indent=4)

# Exibir o JSON no console (opcional)
print(json.dumps(output, indent=4))



image 1/1 /home/gamorab/Desktop/epib-box/model/v1/img1.jpeg: 608x800 1 bola, 1 bomba, 1 bota1, 1 bota2, 1 cx-bike, 1 cx-ferro, 1 cx-magali, 1 cx-minions, 1 cx-xuxa, 1 luz, 1 pote, 8.4ms
Speed: 10.9ms preprocess, 8.4ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)

image 1/1 /home/gamorab/Desktop/epib-box/model/v1/img2.jpeg: 608x800 1 bota1, 1 capacete, 1 cx-bike, 1 cx-ferro, 1 cx-magali, 1 cx-xuxa, 1 luz, 1 pote, 8.1ms
Speed: 2.0ms preprocess, 8.1ms inference, 0.9ms postprocess per image at shape (1, 3, 608, 800)
{
    "image1": {
        "path": "img1.jpeg",
        "objects": {
            "pote": 1,
            "bota2": 1,
            "bola": 1,
            "bota1": 1,
            "bomba": 1,
            "cx-minions": 1,
            "cx-magali": 1,
            "cx-bike": 1,
            "cx-xuxa": 1,
            "luz": 1,
            "cx-ferro": 1
        }
    },
    "image2": {
        "path": "img2.jpeg",
        "objects": {
            "capacete": 1,
        